# OpenFEMA API Tutorial: Part 3 - Paging to get Data
  
## Quick Summary
- This tutorial demonstrates the use of a "paging" technique to retrieve data when more than 1,000 records are found - *it is not a Python language tutorial*
- The OpenFEMA API parameters \\$skip and \\$top are used in conjunction with the record count to iterate through the data
- A [Final Working Example](#Final-Working-Example) demonstrates the paging technique in Python (our GitHub repository contains examples in other languages)
- The next tutorial will cover Getting Dataset Updates

## Overview
In previous tutorials we demonstrated the basic use of the OpenFEMA API and parameters to alter the content and delivery of  data - the fields to return, the sort order, and *what* records to return based on criteria. Other parameters affect data delivery or *how* the data is returned - file format, file names, and the number of records to return at one time. Because OpenFEMA limits the maximum number of records returned per call to 1,000, we need to issue multiple calls using some of these parameters. 
 
The purpose of this notebook is to demonstrate how to retrieve more than 1,000 found records using a technique called "paging" that utilizes the \\$skip parameter, $top parameter, and the "count" value.The examples are presented using Python 3, but it should be easy to translate them to almost any programming language.

See the [OpenFEMA Documentation](https://www.fema.gov/about/openfema/api) for details on the API parameters.

See the [OpenFEMA Code Samples GitHub Repository](https://github.com/FEMA/openfema-samples) for paging examples in various languages.

## Paging Through Data
For performance reasons, only 1,000 records are returned per API endpoint call. If more than 1,000 records exist, it will be necessary to page through the data to retrieve every record. The metadata header returned as part of the data set JSON response will only display the full record count if the \\$inlinecount parameter is used—otherwise, it will have a value of 0. Computer code containing a loop is written to continue making API calls, incrementing the \\$skip parameter accordingly until the number of records retrieved equals the total record count.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Several of the GitHub code examples download CSV files. It is recommended that results be downloaded in a JSON format. This format is native to the OpenFEMA data store, as such data need not be converted by the server thereby improving download performance.
</div>

### Defining and Calling the API Query
Define the endpoint and any parameters. As explained in parts 1 and 2 of the tutorial, the process to call an API endpoint is straightforward - declare a library or module that can facilitate requests to web resources, define the URL/API endpoint, specify parameters, and issue the API call.

<div class="alert alert-block alert-info">
    <b>Tip:</b> 
    <ul>
    <li>If you need the full, unfiltered file, use one of the file download links on the dataset page.
    <li>For very large files, try to limit the data with the \$filter and \$select parameters. This is optional of course.</li>
    <li>Specify a sort order to ensure that the proper records are returned with each call. If the sort is unimportant, use id.</li>
    </ul>
</div>

The previous tutorials used the FemaWebDisasterDeclarations endpoint. This tutorial will use the DisasterDeclarationsSummaries dataset as it contains many more records, allowing us to filter and still find more than 1,000 records.


In [1]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,declarationTitle,state"    # leave this parameter out if you want all fields
filter = "&$filter=state%20eq%20%27LA%27"                                    # for purposes of example, limit to Louisiana

Lets make an initial call using the \\$inlinecount parameter. We want to know how many records match our criteria. This will tell us how many times we must call the endpoint. Why not just include this parameter in our main request for data? As explained in part 2 of the tutorial, \\$inlinecount forces the OpenFEMA server to count all the matching records *for each call*. If working with a large dataset (such as NFIP policies that contains more than 61 million records) where you must issue thousands of calls to retrieve all the data, including $inlinecount in every call will significantly reduce performance.

For this initial call, limit it to 1 record, 1 field, and we don't care about the sort or the return format. Rather than define these parameter values in variables and combine, we will just add them to our URL and filter.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Because we are ignoring the \$select parameter above, we have to add back in our base URL/query separator - the question mark (?). The order in which the parameters are combined is not important.
</div>

In [2]:
# Return 1 record with your criteria to get total record count. Specifying only 1
#   column here to reduce amount of data returned. Need inlinecount to get record count. 
request = urllib.request.urlopen(baseUrl + "?$inlinecount=allpages&$select=id&$top=1" + filter )

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the metadata object
print(json.dumps(jsonData['metadata'], indent=2))

{
  "skip": 0,
  "filter": "state eq 'LA'",
  "orderby": "",
  "select": null,
  "rundate": "2022-11-07T20:57:13.882Z",
  "entityname": "DisasterDeclarationsSummaries",
  "version": "v2",
  "top": 1,
  "count": 2493,
  "format": "json",
  "metadata": true,
  "url": "/api/open/v2/DisasterDeclarationsSummaries?$inlinecount=allpages&$select=id&$top=1&$filter=state%20eq%20%27LA%27"
}


Now that we have a total count of records matching our criteria, we can calculate the number of times we must issue the call to return all of it. We will retrieve the maximum number of records we can (1,000) so we can minimize the number of calls we must make. So, divide by our count by our record limit. Round any fractional amount up or the last call will be missed. We will use the "math" library to help with this.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Because the default number of records returned is 1,000, we don't have to explicitly define it or use the $top parameter. We will define it in this example for clarity.
</div>

In [3]:
import math

# number of records we want to return with each call
top = 1000

# calculate the number of calls we will need to get all of our data (using the maximum of 1000)
recCount = jsonData['metadata']['count']
loopNum = math.ceil(recCount / top)

# print amounts
print("For " + str(recCount) + " records, we will need to issue " + str(loopNum) + " calls")

For 2493 records, we will need to issue 3 calls


Before we define our loop and begin to issue calls, lets take care of a few more items:

 - Define additional parameters:
   - \\$orderby - order is unimportant for the final result in this example, but we should specify one - use id
   - \\$top - as mentioned above, not really necessary because we want the maximum for each call, but for clarity, we will include
   - \\$metadata - lets turn off, otherwise the calls will take longer and we will have to skip it. Since we will be using a jsona format, turning metadata off is optional because specifying this format will suppress the metadata. We will include it in this example for clarity.
   - \\$format - use jsona instead of json to simplify 
 - Open a file so that we can save our data. While this is optional (you could save the data in a variable and then proceed with some sort of manipulation or analysis), the most common use case is to save the data so it can be easily retrieved and used later.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Earlier, we defined the call return type to be jsona. Why? If we use the json format, every call will result in a top level json object (named after the dataset endpoint) containing an array of records. The top level object will have to be skipped for each iteration - all we want are the records. While this is not hard to do in Python, specifying jsona gives us just an array of records.
    
A side effect of this technique is that the final file (of just saved records) will not be recognized as a valid json file for future use (because it really is a json array file). This can be overcome by adding the high level object back in to the file once.
</div>

<div class="alert alert-block alert-info">
    <b>Tip:</b> A csv format may be specified if desired. The code in these examples will have to change slightly. The code examples in the OpenFEMA GitHub repository has examples. 
    
There is a tradeoff with regards to specifying a csv format. Less data must be transmitted because the format is not as verbose, but the OpenFEMA server must work to convert the data from json to csv - potentially increasing the download time. It is also possible to convert the format after it has been downloaded.
</div>

In [4]:
orderby = "&$orderby=id"     # order unimportant to me, so use id
limit = "&$top=1000"         # not needed as the default is 1000 - including for clarity
format = "&$format=jsona"    # lets use an array of json objects - easier
other = "&$metadata=off"     # not needed as jsona suppresses metadata - including for clarity


# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
#   The root json entity is usually the name of the dataset, but you can use any name.
outFile = open("dds_output.json", "a")
outFile.write('{"disasterdeclarationssummaries":[');

Now we are ready to make subsequent calls until all our data has been fetched. With each iteration, skip the records that have already been fetched.

 - Compose a request, skipping those already fetched
 - Continue issuing calls until we have reached our loop count
 - Determine if we are on the last call to terminate the json object if necessary
 
<div class="alert alert-block alert-info">
    <b>Tip:</b> An alternative to saving the data after each call is to accumulate the results in a variable and then save it all at once. This can be faster. This may not be possible due to memory limitations for extremely large datasets.
</div>
 

In [5]:
# Loop and call the API endpoint changing the record start each iteration. The metadata is being
#   suppressed as we no longer need it.
skip = 0
i = 0
while (i < loopNum):
    # By default data is returned as a JSON object, the data set name being the root element. Unless
    #   you extract records as you process, you will end up with 1 distinct JSON object for EVERY 
    #   call/iteration. An alternative is to return the data as JSONA (an array of json objects) with 
    #   no root element - just a bracket at the start and end. This is easier to manipulate.
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    
    # The data is already returned in a JSON format. There is no need to decode and load as a JSON object.
    #   If you want to begin working with and manipulating the JSON, import the json library and load with
    #   something like: jsonData = json.loads(result.decode())

    # Append results to file, trimming off first and last JSONA brackets, adding comma except for last call,
    #   AND root element terminating array bracket and brace to end unless on last call. The goal here is to 
    #   create a valid JSON file that contains ALL the records. This can be done differently.
    if (i == (loopNum - 1)):
        # on the last so terminate the single JSON object
        outFile.write(str(result[1:-1],'utf-8') + "]}")
    else:
        outFile.write(str(result[1:-1],'utf-8') + ",")

    # increment the loop counter and skip value
    i+=1
    skip = i * top

    print("Iteration " + str(i) + " done")

outFile.close()

Iteration 1 done
Iteration 2 done
Iteration 3 done


At this point, all the data has been retrieved and saved. We can verify that all the records were retrieved by opening the file, loading the data, and counting the number of records.

In [6]:
# lets re-open the file and see if we got the number of records we expected
inFile = open("dds_output.json", "r")
my_data = json.load(inFile)
print(str(len(my_data['disasterdeclarationssummaries'])) + " records in file")
inFile.close()


2493 records in file


## Modifications for CSV Format
The following illustrates saving data in a csv format. We changed the \\$format parameter and removed the code to make the result a proper json file. It is necessary to check whether we are on the first pull to ensure we output the field headers only once. 

In [7]:
orderby = "&$orderby=id"     # order unimportant to me, so use id
limit = "&$top=1000"         # not needed as the default is 1000 - including for clarity
format = "&$format=csv"      # lets use csv as our output type
other = "&$metadata=off"     # not needed as csv suppresses metadata - including for clarity


# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
#   The root json entity is usually the name of the dataset, but you can use any name.
outFile = open("dds_output.csv", "a")


# Loop and call the API endpoint changing the record start each iteration. The metadata is being
#   suppressed as we no longer need it.
skip = 0
i = 0
while (i < loopNum):
    # issue call, decode the data, and save to a file
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    csvData = result.decode('utf-8')
    
    # avoid writing the header/fieldnames every time
    if (i == 0):
        # on the first record, so write full output that includes field headers
        outFile.write(csvData)
    else:
        # split off the first row
        outFile.write(csvData.split("\n",1)[1])
    
    # increment the loop counter and skip value
    i+=1
    skip = i * top

    print("Iteration " + str(i) + " done")

outFile.close()

Iteration 1 done
Iteration 2 done
Iteration 3 done


## Final Working Example
Although contrived, this final example applies many of the parameters discussed in this tutorial.  Examples in other languages can be found in the [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples) repository.

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is not meant to be a Python language tutorial. The point is to show the use and combination of OpenFEMA API parameters. There are other, more Pythonic ways that this can be done. If you are writing production quality code, it is recommended that you follow industry best practices - evaluate returned values, add error handling, add logging, proper object cleanup, build for resilience by adding retries if failure, etc.
</div>

In [30]:
# declare a URL handling module
import urllib.request
import json
import math

# define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,declarationTitle,state"    # leave this parameter out if you want all fields
filter = "&$filter=state%20eq%20%27LA%27"                                    # for purposes of example, limit to Louisiana
orderby = "&$orderby=id"     # order unimportant to me, so use id
limit = "&$top=1000"         # not needed as the default is 1000 - including for clarity
format = "&$format=jsona"    # lets use an array of json objects - easier
other = "&$metadata=off"     # not needed as jsona suppresses metadata - including for clarity

# number of records we want to return with each call
top = 1000

# Return 1 record with your criteria to get total record count. Specifying only 1
#   column here to reduce amount of data returned. Need inlinecount to get record count. 
request = urllib.request.urlopen(baseUrl + "?$inlinecount=allpages&$select=id&$top=1" + filter )

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# calculate the number of calls we will need to get all of our data (using the maximum of 1000)
recCount = jsonData['metadata']['count']
loopNum = math.ceil(recCount / top)

# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
#   The root json entity is usually the name of the dataset, but you can use any name.
outFile = open("dds_output.json", "a")
outFile.write('{"disasterdeclarationssummaries":[');

# Loop and call the API endpoint changing the record start each iteration. The metadata is being
#   suppressed as we no longer need it.
skip = 0
i = 0
while (i < loopNum):
    # By default data is returned as a JSON object, the data set name being the root element. Unless
    #   you extract records as you process, you will end up with 1 distinct JSON object for EVERY 
    #   call/iteration. An alternative is to return the data as JSONA (an array of json objects) with 
    #   no root element - just a bracket at the start and end. This is easier to manipulate.
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    
    # The data is already returned in a JSON format. There is no need to decode and load as a JSON object.
    #   If you want to begin working with and manipulating the JSON, import the json library and load with
    #   something like: jsonData = json.loads(result.decode())

    # Append results to file, trimming off first and last JSONA brackets, adding comma except for last call,
    #   AND root element terminating array bracket and brace to end unless on last call. The goal here is to 
    #   create a valid JSON file that contains ALL the records. This can be done differently.
    if (i == (loopNum - 1)):
        # on the last so terminate the single JSON object
        outFile.write(str(result[1:-1],'utf-8') + "]}")
    else:
        outFile.write(str(result[1:-1],'utf-8') + ",")

    # increment the loop counter and skip value
    i+=1
    skip = i * top

outFile.close()

# lets re-open the file and see if we got the number of records we expected
inFile = open("dds_output.json", "r")
my_data = json.load(inFile)
print(str(len(my_data['disasterdeclarationssummaries'])) + " records in file")
inFile.close()

{
  "metadata": {
    "skip": 10,
    "filter": "stateCode eq 'VA'",
    "orderby": "declarationDate DESC",
    "select": null,
    "rundate": "2022-10-04T15:55:07.820Z",
    "entityname": "FemaWebDisasterDeclarations",
    "version": "v1",
    "top": 3,
    "count": 73,
    "format": "json",
    "metadata": true,
    "url": "/api/open/v1/FemaWebDisasterDeclarations?$select=disasterNumber,declarationDate,disasterName,stateCode&$filter=stateCode%20eq%20%27VA%27&$orderby=declarationDate%20desc&$skip=10&$top=3&$format=json&$inlinecount=allpages"
  },
  "FemaWebDisasterDeclarations": [
    {
      "disasterNumber": 4262,
      "declarationDate": "2016-03-07T00:00:00.000Z",
      "disasterName": "SEVERE WINTER STORM AND SNOWSTORM",
      "stateCode": "VA"
    },
    {
      "disasterNumber": 4092,
      "declarationDate": "2012-11-26T00:00:00.000Z",
      "disasterName": "HURRICANE SANDY",
      "stateCode": "VA"
    },
    {
      "disasterNumber": 3359,
      "declarationDate": "2012-10-2

### Where to go Next
OpenFEMA API_Tutorial_Part_4_GettingDatasetUpdates

## Other Resources
- [OpenFEMA Homepage](https://www.fema.gov/open)
- [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api)
- [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples)